In [ ]:
# !pip install pororo -q

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

from datetime import datetime, timedelta
import time
from tqdm import trange

In [ ]:
review_list= 'https://gift.kakao.com/a/v2/review/products/1152525?page=0&sortProperty=LATEST&size=20'

#### 📄제품 LIst 확보

In [ ]:
# URL 예시
main = 'https://gift.kakao.com/'
product_list = 'https://gift.kakao.com/a/v1/pages/productGroups/collections?page=1&size=50&brandIds=5627&filteringSoldOut=true&sortProperty=RELEASE&sortDir=DESC' #json
review_list  = 'https://gift.kakao.com/a/v2/review/products/2942926?page=0&sortProperty=LATEST&size=20' #json

In [ ]:
url_list = 'https://gift.kakao.com/a/v1/pages/productGroups/collections?page=1&size=50&brandIds=5627&filteringSoldOut=true&sortProperty=RELEASE&sortDir=DESC'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.141 Safari/537.36'}
KaPresent = requests.get(url_list, headers=headers).json()

In [ ]:
len(KaPresent['items'])

49

In [ ]:
KaPresent['items'][6]

{'brandId': 5627,
 'brandName': '매일유업(식품)',
 'discountPercent': 0,
 'discountedPrice': 46900,
 'displayName': '[매일유업]매일야채 토마토의힘/당근의 힘 125ml 48팩',
 'giftItemId': 4759974,
 'imageUrl': 'https://st.kakaocdn.net/product/gift/product/20220328105207_175bab8a82c94ba5b73321b6eae90518.jpg',
 'name': '[매일유업]매일야채 토마토의힘/당근의 힘 125ml 48팩',
 'productId': 4535950,
 'salePrice': 46900,
 'soldOut': False,
 'stamp': 'ON_SALE',
 'talkStar': False}

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.141 Safari/537.36'}
url_list = ['https://gift.kakao.com/a/v1/pages/productGroups/collections?page=1&size=50&brandIds=5627&filteringSoldOut=true&sortProperty=RELEASE&sortDir=DESC']
code = []
for url_temp in url_list:
    KaPresent = requests.get(url_temp, headers=headers).json()
    for i in range(0,len(KaPresent['items'])):
        product = KaPresent['items'][i]['displayName']
        product_code = KaPresent['items'][i]['productId']
        rev_url= 'https://gift.kakao.com/a/v2/review/products/' + str(product_code) + '?page=0&sortProperty=LATEST&size=20'
        code.append([product, rev_url])

In [ ]:
# code

#### 🅰제품 리뷰 수집

In [ ]:
prod, url_temp2 = code[1]
KaPresent2 = requests.get(url_temp2, headers=headers).json()
len(KaPresent2['reviewList']['contents'])

1

In [ ]:
url_temp2

'https://gift.kakao.com/a/v2/review/products/4944395?page=0&sortProperty=LATEST&size=20'

{'reviewList': {'condition': {'page': 0, 'size': 20},
  'contents': [{'attachments': [],
    'content': '처음 마셔봤는데  좋네요',
    'displayStateInfo': {'displayStatus': 'OPEN',
     'displayedAt': '20220624200027',
     'reviewEditable': True},
    'id': '62b5994b38a77820deae98a7',
    'pick': False,
    'product': {'id': 4944395,
     'option': '더그레잇티 콤부차: 오리지널6팩+베리리치6팩(총12팩)',
     'optionId': 319930454,
     'rating': 4},
    'reactions': {'counts': [{'count': 0, 'type': 'FUNNY'},
      {'count': 0, 'type': 'IMPRESSED'},
      {'count': 0, 'type': 'UPSET'},
      {'count': 0, 'type': 'WANTED'}],
     'totalCount': 0},
    'tag1': '응원',
    'tag2': '친구',
    'writer': {'me': False,
     'nickname': '행복한 튜브',
     'profileImageUrl': 'https://gift-s.kakaocdn.net/dn/gift/gift/tube.png',
     'showContentUtilizationWithdrawal': False}}],
  'last': True,
  'totalCount': 1},
 'reviewNotAllowedReasons': [],
 'reviewable': True}

In [ ]:
date_start = (datetime.today()+ timedelta(hours=9) - timedelta(days=3)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
date_end   = (datetime.today()+ timedelta(hours=9) - timedelta(days=0)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
for prod, url_temp2 in code:
    KaPresent2 = requests.get(url_temp2, headers=headers).json()
    for i in range(len(KaPresent2['reviewList']['contents'])):
        reg_date= KaPresent2['reviewList']['contents'][i]['displayStateInfo']['displayedAt'][:8]
        reg_time= KaPresent2['reviewList']['contents'][i]['displayStateInfo']['displayedAt'][9:]
        DATE1   = datetime.strptime(reg_date, "%Y%m%d").strftime('%Y-%m-%d')
        DATE2   = datetime.strptime(reg_time, "%H%M%S").strftime('%H:%M:%S')
        DATE    = DATE1 + ' ' +  DATE2
        if date_start <= DATE < date_end :
            USER = KaPresent2['reviewList']['contents'][i]['id']
            CHANNEL = '카카오 선물하기'
            try:
                PRODUCT = KaPresent2['reviewList']['contents'][i]['product']['option']
                if PRODUCT == '':
                    PRODUCT = prod
            except:
                PRODUCT = prod
            gubun_code = KaPresent2['reviewList']['contents'][i]['product']['id']
            REVIEW  = KaPresent2['reviewList']['contents'][i]['content']
            SCORE   = KaPresent2['reviewList']['contents'][i]['product']['rating']
            rows_to_insert = [(USER, CHANNEL, PRODUCT,  REVIEW, int(SCORE), DATE, gubun_code)]
            print(rows_to_insert)

In [ ]:
date_start = (datetime.today()+ timedelta(hours=9) - timedelta(days=3)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
date_end   = (datetime.today()+ timedelta(hours=9) - timedelta(days=0)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
for prod, url_temp2 in code:
    KaPresent2 = requests.get(url_temp2, headers=headers).json()
    for i in range(len(KaPresent2['reviewList']['contents'])):
        reg_date= KaPresent2['reviewList']['contents'][i]['displayStateInfo']['displayedAt'][:8]
        reg_time= KaPresent2['reviewList']['contents'][i]['displayStateInfo']['displayedAt'][9:]
        DATE1   = datetime.strptime(reg_date, "%Y%m%d").strftime('%Y-%m-%d')
        DATE2   = datetime.strptime(reg_time, "%H%M%S").strftime('%H:%M:%S')
        DATE    = DATE1 + ' ' +  DATE2
        if date_start <= DATE < date_end :
            USER = KaPresent2['reviewList']['contents'][i]['id']
            CHANNEL = '카카오 선물하기'
            PRODUCT = prod
            REVIEW  = KaPresent2['reviewList']['contents'][i]['content']
            SCORE   = KaPresent2['reviewList']['contents'][i]['product']['rating']
            rows_to_insert = [(USER, CHANNEL, PRODUCT,  REVIEW, int(SCORE), DATE, '리뷰')]
            print(rows_to_insert)

[('62ce9c03d3935d3a78f34f57', '카카오 선물하기', '"생일축하해" 상하목장 유기농 아이스크림 파인트 밀크/딸기 474ml 2개', '배송 포장 상태 아주 좋아요. ^^', 4, '2022-07-13 09:18:43', '리뷰')]
[('62ceaecba447ce25286c3c6b', '카카오 선물하기', '[매일유업] 매일두유 고단백 190ml 24팩', '막내여동생이 나이들어가는  언니인 나에게 단백질보충용으로 보내준 의미있는 선물.... 감동입니다', 4, '2022-07-13 03:08:51', '리뷰')]
[('62cec567a447ce25286c3cea', '카카오 선물하기', '[매일유업]상하목장 얼려먹는 아이스크림 초코/밀크,주스 망고 85ml 24개입', '아직 먹어보진 않았지만 유듀브에 먹방으로 뜨길래 마침 언니의 선물덕에 접해봤습니다~^^\n얼려먹어야 하니 내일이나 맛볼수 있네요ㅎ', 4, '2022-07-13 21:51:09', '리뷰')]
[('62cd9a63a447ce25286c3603', '카카오 선물하기', '[매일유업]상하목장 얼려먹는 아이스크림 초코/밀크,주스 망고 85ml 24개입', '어이스크림 처돌이는 너무 좋습니다 실온 보관이라 냉동실 공간 차지 안해서 좋아요', 4, '2022-07-13 05:09:31', '리뷰')]
[('62ce577f77ec47603e9db1bd', '카카오 선물하기', '[매일유업] 피크닉 사과/청포도 200ml 24팩', '전 여자친구가 좋아한다고 사줬는데 사준 당일에 헤어져서 피크닉 말고 술만 먹을 거 같네요', 4, '2022-07-13 04:26:23', '리뷰')]


In [ ]:
def kakaopresent_review(t1):
    date_start = (datetime.today()+ timedelta(hours=9) - timedelta(days=1)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
    date_end   = (datetime.today()+ timedelta(hours=9) - timedelta(days=0)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
    for prod, url_temp2 in code:
        KaPresent2 = requests.get(url_temp2, headers=headers).json()
        for i in range(len(KaPresent2['reviewList']['contents'])):
            reg_date= KaPresent2['reviewList']['contents'][i]['displayStateInfo']['displayedAt'][:8]
            reg_time= KaPresent2['reviewList']['contents'][i]['displayStateInfo']['displayedAt'][9:]
            DATE1   = datetime.strptime(reg_date, "%Y%m%d").strftime('%Y-%m-%d')
            DATE2   = datetime.strptime(reg_time, "%H%M%S").strftime('%H:%M:%S')
            DATE    = DATE1 + ' ' +  DATE2
            if date_start <= DATE < date_end :
                USER = KaPresent2['reviewList']['contents'][i]['id']
                CHANNEL = '카카오 선물하기'
                PRODUCT = prod
                REVIEW  = KaPresent2['reviewList']['contents'][i]['content'].replace('\n', ' ')
                SCORE   = KaPresent2['reviewList']['contents'][i]['product']['rating']
                if int(SCORE) >= 3:
                    SCORE = int(SCORE) + 1
                rows_to_insert = [(USER, CHANNEL, PRODUCT,  REVIEW, int(SCORE), DATE, '리뷰')]
                print(rows_to_insert)

In [ ]:
kakaopresent_review('')

[('62ce9c03d3935d3a78f34f57', '카카오 선물하기', '"생일축하해" 상하목장 유기농 아이스크림 파인트 밀크/딸기 474ml 2개', '배송 포장 상태 아주 좋아요. ^^', 5, '2022-07-13 09:18:43', '리뷰')]
[('62ceaecba447ce25286c3c6b', '카카오 선물하기', '[매일유업] 매일두유 고단백 190ml 24팩', '막내여동생이 나이들어가는  언니인 나에게 단백질보충용으로 보내준 의미있는 선물.... 감동입니다', 5, '2022-07-13 03:08:51', '리뷰')]
[('62cec567a447ce25286c3cea', '카카오 선물하기', '[매일유업]상하목장 얼려먹는 아이스크림 초코/밀크,주스 망고 85ml 24개입', '아직 먹어보진 않았지만 유듀브에 먹방으로 뜨길래 마침 언니의 선물덕에 접해봤습니다~^^ 얼려먹어야 하니 내일이나 맛볼수 있네요ㅎ', 5, '2022-07-13 21:51:09', '리뷰')]
[('62cd9a63a447ce25286c3603', '카카오 선물하기', '[매일유업]상하목장 얼려먹는 아이스크림 초코/밀크,주스 망고 85ml 24개입', '어이스크림 처돌이는 너무 좋습니다 실온 보관이라 냉동실 공간 차지 안해서 좋아요', 5, '2022-07-13 05:09:31', '리뷰')]
[('62ce577f77ec47603e9db1bd', '카카오 선물하기', '[매일유업] 피크닉 사과/청포도 200ml 24팩', '전 여자친구가 좋아한다고 사줬는데 사준 당일에 헤어져서 피크닉 말고 술만 먹을 거 같네요', 5, '2022-07-13 04:26:23', '리뷰')]


In [ ]:
def kakaopresent_review(t1):
    date_start = (datetime.today()+ timedelta(hours=9) - timedelta(days=1)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
    date_end   = (datetime.today()+ timedelta(hours=9) - timedelta(days=0)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
    for prod, url_temp2 in code:
        KaPresent2 = requests.get(url_temp2, headers=headers).json()
        for i in range(len(KaPresent2['reviewList']['contents'])):
            reg_date= KaPresent2['reviewList']['contents'][i]['displayStateInfo']['displayedAt'][:8]
            reg_time= KaPresent2['reviewList']['contents'][i]['displayStateInfo']['displayedAt'][9:]
            DATE1   = datetime.strptime(reg_date, "%Y%m%d").strftime('%Y-%m-%d')
            DATE2   = datetime.strptime(reg_time, "%H%M%S").strftime('%H:%M:%S')
            DATE    = DATE1 + ' ' +  DATE2
            if date_start <= DATE < date_end :
                USER = KaPresent2['reviewList']['contents'][i]['id']
                CHANNEL = '카카오 선물하기'
                PRODUCT = prod
                REVIEW  = KaPresent2['reviewList']['contents'][i]['content'].replace('\n', ' ')
                SCORE   = KaPresent2['reviewList']['contents'][i]['product']['rating']
                if int(SCORE) >= 3:
                    SCORE = int(SCORE) + 1
                rows_to_insert = [(USER, CHANNEL, PRODUCT,  REVIEW, int(SCORE), DATE, '리뷰')]
                print(rows_to_insert)
                # client.insert_rows(Table, rows_to_insert)
    return f'Hello World!'

#### 🆗제품 리뷰 수집(API)

In [ ]:
# main.py
import pandas as pd
from datetime import datetime, timedelta
import requests
import time
import re
from google.cloud import bigquery
from bs4 import BeautifulSoup

# client = bigquery.Client()
# project = 'boxwood-chassis-356100'
# dataset_id = 'review'
# table_id = 'review_all_test'
# table_ref = client.dataset(dataset_id).table(table_id)
# Table = client.get_table(table_ref)  # API request

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.141 Safari/537.36'}
url_list = ['https://gift.kakao.com/a/v1/pages/productGroups/collections?page=1&size=50&brandIds=5627&filteringSoldOut=true&sortProperty=RELEASE&sortDir=DESC',
            'https://gift.kakao.com/a/v1/pages/productGroups/collections?page=1&size=50&brandIds=13586&filteringSoldOut=true&sortProperty=RELEASE&sortDir=DESC']
code = []
for url_temp in url_list:
    KaPresent = requests.get(url_temp, headers=headers).json()
    for i in range(0,len(KaPresent['items'])):
        product = KaPresent['items'][i]['displayName']
        product_code = KaPresent['items'][i]['productId']
        rev_url= 'https://gift.kakao.com/a/v2/review/products/' + str(product_code) + '?page=0&sortProperty=LATEST&size=20'
        code.append([product, rev_url])

def kakaopresent_review(t1):
    date_start = (datetime.today()+ timedelta(hours=9) - timedelta(days=1)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
    date_end   = (datetime.today()+ timedelta(hours=9) - timedelta(days=0)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
    for prod, url_temp2 in code:
        KaPresent2 = requests.get(url_temp2, headers=headers).json()
        for i in range(len(KaPresent2['reviewList']['contents'])):
            reg_date= KaPresent2['reviewList']['contents'][i]['displayStateInfo']['displayedAt'][:8]
            reg_time= KaPresent2['reviewList']['contents'][i]['displayStateInfo']['displayedAt'][9:]
            DATE1   = datetime.strptime(reg_date, "%Y%m%d").strftime('%Y-%m-%d')
            DATE2   = datetime.strptime(reg_time, "%H%M%S").strftime('%H:%M:%S')
            DATE    = DATE1 + ' ' +  DATE2
            if date_start <= DATE < date_end :
                USER = KaPresent2['reviewList']['contents'][i]['id']
                CHANNEL = '카카오 선물하기'
                PRODUCT = prod
                REVIEW  = KaPresent2['reviewList']['contents'][i]['content'].replace('\n', ' ')
                SCORE   = KaPresent2['reviewList']['contents'][i]['product']['rating']
                if int(SCORE) >= 3:
                    SCORE = int(SCORE) + 1
                rows_to_insert = [(USER, CHANNEL, PRODUCT,  REVIEW, int(SCORE), DATE, '리뷰')]
                print(rows_to_insert)
                # client.insert_rows(Table, rows_to_insert)
    return f'Hello World!'

In [ ]:
kakaopresent_review('')

'Hello World!'